In [13]:
import json

post_list = []
with open('data/facebook/고려대학교 대나무숲/00001.json', 'r') as fp:
    buf = ''
    for line in fp.readlines():
        line = line.strip()

        buf += line
        if line == '}':
            d = json.loads(buf)
            buf = ''
            
            post_list.append(d)

In [14]:
post_list[0]

{'html': '<div class="_5pcr userContentWrapper" data-ft=\'{"tn":"-R"}\' style=""><div class="_1dwg _1w_m _q7o"><div class="_4r_y" id="u_t_c"><div class="_6a uiPopover _5pbi _cmw _b1e _1wbl" id="u_t_d"><a aria-expanded="false" aria-haspopup="true" aria-label="소식 옵션" class="_4xev _p" data-testid="post_chevron_button" href="#" id="u_t_p" rel="toggle" role="button"></a></div></div><div><div class="u_1jru-ci9ew c_1jru-cbygh clearfix"><div class="clearfix p_1jru-ciael"><a aria-hidden="true" class="_5pb8 q_1jru-cbygm _8o _8s lfloat _ohe" data-ft=\'{"tn":"\\u003C"}\' data-hovercard="/ajax/hovercard/page.php?id=206910909512230&amp;extragetparams=%7B%22__tn__%22%3A%22%2Cd%5Cu003C-R-R%22%2C%22eid%22%3A%22ARAMOqV0xpgRkTOtGZoCerw7WEqBpkMFTj-pxZrppZbxe0mEMHmkLXld9sf9tRYk8y57tT8ccdh0WGCh%22%2C%22hc_ref%22%3A%22ARRlkBRacYCQtdKfCJwpTCO3nqonLQAKWMKSHIvPTOzLxUrvSH06N05_eHn0UCUaPfw%22%7D" data-hovercard-prefer-more-content-show="1" data-hovercard-referer="ARRlkBRacYCQtdKfCJwpTCO3nqonLQAKWMKSHIvPTOzLxUrvSH

In [52]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(post_list[0]['html'])

reply_list = soup.find('div', {'data-testid': 'UFI2CommentsList/root_depth_0'})
# for reply in reply.find_all('div', {'data-testid': 'UFI2Comment/body'}):
for reply in reply_list.find_all('div', {'data-testid': 'UFI2Comment/root_depth_0'}):
#     print(reply, '\n\n')
    item = {}

    tag = reply.find({'data-testid': 'UFI2Comment/body'})

    link = [v.get_text() for v in tag.find_all('a')]
    [v.extract() for v in tag.find_all('a')]

    if len(link) > 1:
        item['reply_to'] = link[-1]

    item['message'] = str(tag)
    
    print(item)
    
#     for tag in reply.find_all():
#         attr = 'alt'
#         if tag.has_attr(attr):
#             item['user_name'] = tag[attr]
            
#         attr = 'data-testid'
#         if tag.has_attr(attr) and tag[attr] == 'UFI2Comment/body':
#             link = [v.get_text() for v in tag.find_all('a')]
#             [v.extract() for v in tag.find_all('a')]
            
#             if len(link) > 1:
#                 item['reply_to'] = link[-1]
                
#             item['message'] = str(tag)
            
#         attr = 'data-tooltip-content'
#         if tag.has_attr(attr) and tag[attr].find('숨기기') < 0:
#             item['date'] = tag[attr]
            
#         print(item)
        
    print('\n\n\n')        

AttributeError: 'NoneType' object has no attribute 'find_all'

In [20]:
#!./venv/bin/python3
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import requests

from time import sleep, time
from datetime import datetime

In [24]:
access_token = 'EAACBRrCNUOwBAABVzTeRCtaGlETI3jz9n7uNEkSOZBiCPhF8I14g4VLNCZC5AQ9rxKR2b4j8j15vTF8Pi8wmiSSN8zS6Srw2q0eZAD7ZCLVW8XFySHTuxagCYArYCtrGUv6xKsBGa6un9fYMUrQ1Tfow9TZBBjKkcNZC05J1wjVP39f50pR1M3Nw2wCllrFJnQVblTBEW3MAZDZD'

fields= 'id,name,posts,groups,likes'
url = 'https://graph.facebook.com/v4.0/me?fields={fields}&access_token={access_token}'.format(fields=fields, access_token=access_token)

# url = 'https://graph.facebook.com/v4.0/{page_id}/feed/?fields={fields}&access_token={access_token}'.format(fields=fields, access_token=access_token)


In [25]:
resp = requests.get(url)
resp

<Response [200]>

In [26]:
resp.json()

{'id': '1500855749963737', 'name': '박은진'}